<a href="https://colab.research.google.com/github/MHoffmannAC/Data-Science-Primer/blob/main/Hangman_by_MHoffmann.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hangman Game

### by M. Hoffmann

## Full code definitions

In [1]:
# @title Initial setup of environment

### colab-specific stuff and basic imports
#from google.colab import output
from IPython.display import display, clear_output
from time import sleep
import random

### for English I use random words and sentences
!pip install wonderwords
import wonderwords

### for German and 1984 I use lists from URLs:
import urllib.request

### pyfiglet for some visual adjustments
#!pyfiglet --list_fonts  # in case you want different fonts
!pip install pyfiglet
from pyfiglet import Figlet

# for hidden user input:
from getpass import getpass

# might be usefull at some point:
#!pip install translate
#from translate import Translator

In [2]:
# @title Define basic functions (run_game, evaluate_guess, cleaned_solution, random_from_web)

# the main code for a single game of hangman
def run_game(settings):
  guesses_remaining=6
  # generate solution via function calls defined in language dictionary
  rand_func, rand_args, rand_kwargs = lang_dict[settings["language"]]['difficulties'][settings["difficulty"]]['function']
  solution = cleaned_solution(rand_func(*rand_args, **rand_kwargs).upper())
  guessed_word_so_far = "_ " * len(solution)
  possible_guesses=["A B C D E F G H I", "J K L M N O P Q R", "S T U V W X Y Z"]
  # make sure to work fine for sentences and not only single words
  if(" " in solution):
    for i in range(len(solution)):
      if(solution[i] == " "):
        guessed_word_so_far=guessed_word_so_far[:2*i]+" "+guessed_word_so_far[2*i+1:]
  # create some empty lists to keep track of guesses
  # could be one combined list for now since no additional functionality
  wrong_guesses = []
  correct_guesses = []
  letters_to_guess = guessed_word_so_far.count("_")
  # loop over user input while showing game board.
  # stop if solution found or no more guesses left
  while(guesses_remaining>0 and letters_to_guess > 0):
    user_guess = show_game_screen(guessed_word_so_far, guesses_remaining, possible_guesses, settings)
    wrong_guesses, correct_guesses, guesses_remaining, guessed_word_so_far, possible_guesses = evaluate_guess(user_guess, solution, wrong_guesses, correct_guesses, guesses_remaining, guessed_word_so_far, possible_guesses, settings['language'])
    letters_to_guess = guessed_word_so_far.count("_")
  # return game outcome
  if(guesses_remaining==0):
    return False, solution
  else:
    return True, solution

# cleaning the solution string since format from external might not be suitable for hangman
def cleaned_solution(solution):
  return (
        solution
        .replace('Ö', 'OE')
        .replace('Ü', 'UE')
        .replace('Ä', 'AE')
        .replace(",", "")
        .replace(".", "")
        .replace("!", "")
        .replace("?", "")
        .replace(":", "")
        .replace(";", "")
        .replace("-", "")
        .replace("_", " ")
        .replace("\"", "")
        .replace(")","")
        .replace("(", "")
        .replace("\n", "")
        .strip()
        )

# ---------------------------------------

# check user guess for validity, novelty, correctness etc.
def evaluate_guess(user_guess, solution, wrong_guesses, correct_guesses, guesses_remaining, guessed_word_so_far, possible_guesses, language):
  print("")
  if(user_guess=="QUIT!"): # mainly for debugging, the user does not need to know that option
    guesses_remaining=0
  elif(not (  (len(user_guess)==1 and user_guess.isalpha())
               or
              (len(user_guess)==len(solution))
              )):
    print(lang_dict[language]['msg_allowed_input'])
    sleep(2)
  elif(user_guess in wrong_guesses or user_guess in correct_guesses):
    print(lang_dict[language]['msg_repeat'])
    sleep(2)
  elif(len(user_guess)==len(solution)):
    if(user_guess==solution):
      guessed_word_so_far=solution
    else:
      print(lang_dict[language]['msg_not_solution'].format(user_guess=user_guess))
      sleep(2)
      guesses_remaining-=1
  else:
    if(user_guess in solution):
      for i in range(len(solution)):
        if(solution[i] == user_guess):
          guessed_word_so_far=guessed_word_so_far[:2*i]+user_guess+guessed_word_so_far[2*i+1:]
      correct_guesses.append(user_guess)
      print(lang_dict[language]['msg_correct_lett'].format(user_guess=user_guess))
      sleep(2)
    else:
      wrong_guesses.append(user_guess)
      guesses_remaining-=1
      print(lang_dict[language]['msg_wrong_lett'].format(user_guess=user_guess))
      sleep(2)
  for i in range(len(possible_guesses)):
    possible_guesses[i] = possible_guesses[i].replace(user_guess," ")
  return wrong_guesses, correct_guesses, guesses_remaining, guessed_word_so_far, possible_guesses

def random_from_web(page, minlen = 3):
  return random.choice([line.decode('utf-8') for line in urllib.request.urlopen(page) if (len(line)>=minlen)])

In [3]:
# @title Define console outputs (welcome (including defining settings), game, result)

# welcome screen to choose difficulty and/or change language
def show_welcome_screen(settings):
  while(True):
    clear_output(wait=True)
    print(Figlet(font=settings['figlet']).renderText("HANGMAN").rstrip())
    print("")
    print("\033[4m" + lang_dict[settings['language']]['msg_avail_diff'] + "\033[0m")
    for i in lang_dict[settings['language']]['difficulties'].keys():
      print(i + ": " + lang_dict[settings['language']]['difficulties'][i]['label'])
    print("")
    print("\033[4m" + lang_dict[settings['language']]['msg_avail_lang'] + "\033[0m")
    for i in lang_dict.keys():
      if(i == settings['language']):
        print("\033[1m" + i + ": " + lang_dict[i]['lang_full'] + "\033[0m")
      else:
        print(i + ": " + lang_dict[i]['lang_full'])
    print("")
    print("\033[4m" + lang_dict[settings['language']]['msg_avail_figs'] + "\033[0m")
    for i in figures.keys():
      if(i == settings['figure']):
        print("\033[1m" + i + ": " + lang_dict[settings['language']]['fig_texts'][i] + "\033[0m")
      else:
        print(i + ": " + lang_dict[settings['language']]['fig_texts'][i])
    print("")
    user_input = input(lang_dict[settings['language']]['msg_inp_diff'])
    if(user_input in lang_dict[settings['language']]['difficulties'].keys()):
      settings["difficulty"] = user_input
      return True       # leave the function after choosing difficulty
    elif(user_input in lang_dict.keys()):
      settings['language'] = user_input
    elif(user_input in figures.keys()):
      settings['figure'] = user_input
    elif(user_input == "quit!"):
      return False

# ---------------------------------------

def show_game_screen(word, remaining, letters, settings):
  while(True):
    clear_output(wait=True)
    space_needed=max(len(lang_dict[settings['language']]['msg_guesses_left'][1].format(num_guesses=remaining))+10,len(word)+5)
    print("")
    print(" "*space_needed + figures_lines[settings['figure']][6-remaining][0])
    print(word + " "*(space_needed-len(word)) + figures_lines[settings['figure']][6-remaining][1])
    print(" "*space_needed + figures_lines[settings['figure']][6-remaining][2])
    print(lang_dict[settings['language']]['msg_avail_lett'] + " "*(space_needed-len(lang_dict[settings['language']]['msg_avail_lett'])) + figures_lines[settings['figure']][6-remaining][3])
    print(letters[0] + " "*(space_needed-len(letters[0])) + figures_lines[settings['figure']][6-remaining][4])
    print(letters[1] + " "*(space_needed-len(letters[1])) + figures_lines[settings['figure']][6-remaining][5])
    print(letters[2] + " "*(space_needed-len(letters[2])) + figures_lines[settings['figure']][6-remaining][6])
    print(" "*space_needed + figures_lines[settings['figure']][6-remaining][7])
    if(remaining>1):
      print(lang_dict[settings['language']]['msg_guesses_left'][1].format(num_guesses=remaining))
    else:
      print(lang_dict[settings['language']]['msg_guesses_left'][0])
    print("")
    user_input=input(lang_dict[settings['language']]['msg_choose_lett'])
    return user_input.upper()
    break

# ---------------------------------------

def show_result_screen(successful, solution, settings):
  while(True):
    clear_output(wait=True)
    if(successful):
      print("")
      print("\033[32m"+Figlet(font=settings['figlet']).renderText(lang_dict[settings['language']]['header_win']).rstrip()+"\033[0m")
      print("")
      print(lang_dict[settings['language']]['msg_success'].format(solution=solution))
    else:
      print("\033[31m"+Figlet(font=settings['figlet']).renderText(lang_dict[settings['language']]['header_lost']).rstrip()+"\033[0m")
      print(figures[settings['figure']][6])
      print("")
      print(lang_dict[settings['language']]['msg_reveal'].format(solution=solution))
    print("")
    user_input = input(lang_dict[settings['language']]['msg_restart_quit'])
    if(user_input == "r"):
      return True
    elif(user_input == "q"):
      return False
    else:
      continue



In [4]:
# @title Define hangmen figures (and others) for screen outputs

figures = {

    'hangman': ['''
  +---+
  |   |
      |
      |
      |
      |
=========''', '''
  +---+
  |   |
  O   |
      |
      |
      |
=========''', '''
  +---+
  |   |
  O   |
  |   |
      |
      |
=========''', '''
  +---+
  |   |
  O   |
 /|   |
      |
      |
=========''', '''
  +---+
  |   |
  O   |
 /|\  |
      |
      |
=========''', '''
  +---+
  |   |
  O   |
 /|\  |
 /    |
      |
=========''', '''
            +---+
            |   |
            O   |
           /|\  |
           / \  |
                |
          ========='''],


    'flowers': [r'''
         wWWWw               wWWWw
   vVVVv (___) wWWWw         (___)  vVVVv
   (___)  ~Y~  (___)  vVVVv   ~Y~   (___)
    ~Y~   \|    ~Y~   (___)    |/    ~Y~
    \|   \ |/   \| /  \~Y~/   \|    \ |/
   \\|// \\|// \\|/// \\|//  \\|// \\\|///
  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^''',
r'''
         wWWWw
   vVVVv (___) wWWWw                vVVVv
   (___)  ~Y~  (___)  vVVVv         (___)
    ~Y~   \|    ~Y~   (___)    |     ~Y~
    \|   \ |/   \| /  \~Y~/    |    \ |/
   \\|// \\|// \\|/// \\|//  \\|// \\\|///
  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^''',
r'''
         wWWWw
         (___) wWWWw                vVVVv
          ~Y~  (___)  vVVVv         (___)
          \|    ~Y~   (___)    |     ~Y~
     |   \ |/   \| /  \~Y~/    |    \ |/
   \\|// \\|// \\|/// \\|//  \\|// \\\|///
  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^''',
r'''
         wWWWw
         (___) wWWWw                vVVVv
          ~Y~  (___)                (___)
          \|    ~Y~            |     ~Y~
     |   \ |/   \| /           |    \ |/
   \\|// \\|// \\|/// \\|//  \\|// \\\|///
  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^''',
r'''

               wWWWw                vVVVv
               (___)                (___)
           |    ~Y~            |     ~Y~
     |     |    \| /           |    \ |/
   \\|// \\|// \\|/// \\|//  \\|// \\\|///
  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^''',
r'''

               wWWWw
               (___)
           |    ~Y~            |
     |     |    \| /           |      |
   \\|// \\|// \\|/// \\|//  \\|// \\\|///
  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^''',
r'''



           |                   |
     |     |     |             |      |
   \\|// \\|// \\|/// \\|//  \\|// \\\|///
  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^''',
],
    'chase': [r'''

  ____□_
-/|_||_\`.__---------------------------- __o ----
( POLICE _  \ _    ___    ___    ___   _ \<_   __
=`-(_)--(_)-'                         (_)/(_)
-------------------------------------------------
''',
r'''

       ____□_
------/|_||_\`.__----------------------- __o ----
___  ( POLICE _  \ ___    ___    ___   _ \<_   __
     =`-(_)--(_)-'                    (_)/(_)
-------------------------------------------------
''',
r'''

            ____□_
-----------/|_||_\`.__------------------ __o ----
___    ___( POLICE _  \   ___    ___   _ \<_   __
          =`-(_)--(_)-'               (_)/(_)
-------------------------------------------------
''',
r'''

                 ____□_
----------------/|_||_\`.__------------- __o ----
___    ___    _( POLICE _  \_    ___   _ \<_   __
               =`-(_)--(_)-'          (_)/(_)
-------------------------------------------------
''',
r'''

                      ____□_
---------------------/|_||_\`.__-------- __o ----
___    ___    ___   ( POLICE _  \___   _ \<_   __
                    =`-(_)--(_)-'     (_)/(_)
-------------------------------------------------
''',
r'''

                           ____□_
--------------------------/|_||_\`.__--- __o ----
___    ___    ___    ___ ( POLICE _  \ _ \<_   __
                         =`-(_)--(_)-'(_)/(_)
-------------------------------------------------
''',
r'''      _________________________
         ||   ||     ||   ||
         ||   ||, , ,||   ||
         ||  (||/|/(\||/  ||
         ||  ||| _'_`|||  ||
         ||   || o o ||   ||
         ||  (||  - `||)  ||
         ||   ||  =  ||   ||
         ||   ||\___/||   ||
         ||___||) , (||___||
        /||---||-\_/-||---||\
       / ||--_||_____||_--|| \
      (_(||)-| S123-45 |-(||)_)
''']
}


# I need it row-wise for my outputs, so let's do that here:
figures_lines = {}
for figure in figures:
  figures_lines[figure] = []
  for i in range(len(figures[figure])):
    figures_lines[figure].append(figures[figure][i].split("\n"))

In [9]:
# @title Define supported languages and their behaviors

lang_dict = {

    'en': {
        'lang_full': "english",
        'msg_avail_diff': "Available difficulties:",
        'difficulties': {
            '1': {
                'function': (random.choice,[["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]],{}),
                'label': "   Words - easy   (handpicked)"
            },
            '2': {
                'function': (wonderwords.RandomWord().word, [], {'include_parts_of_speech': ["nouns"]}),
                'label': "   Words - medium (random nouns)"
            },
            '3': {
                'function': (wonderwords.RandomWord().word, [], {}),
                'label': "   Words - hard   (random words)"
            },
            '4': {
                'function': (wonderwords.RandomSentence().sentence, [], {}),
                'label': "   Sentence - hard (often gibberish)"
            },
            '5': {
                'function': (getpass, ["Choose your solution word or sentence:  "], {}),
                'label': "   Use user input as solution"
            },
            '1984': {
                'function': (random_from_web, ["https://raw.githubusercontent.com/MHoffmannAC/Data-Science-Primer/main/doublethink"], {}),
                'label': "Doublethink sentences"
            }
        },
        'msg_inp_diff': "Enter difficulty to start a new game\nor choose different language or illustration\n",
        'msg_avail_lett': "Available letters:",
        'msg_avail_lang': "Available languages:",
        'msg_avail_figs': "Available illustrations:",
        'msg_guesses_left': ["You have one wrong guess left",
                             "You have {num_guesses} wrong guesses left"],
        'msg_choose_lett': "Please choose a letter or guess the solution   ",
        'msg_allowed_input': "Please enter a single letter or the full solution",
        'msg_repeat': "You already guessed this letter",
        'msg_not_solution': "Sorry, {user_guess} is not the solution",
        'msg_correct_lett': "Congratulations, {user_guess} is in the solution",
        'msg_wrong_lett': "Sorry, {user_guess} is not in the solution",
        'msg_success': "You successfully guessed the solution {solution}",
        'msg_reveal': "The correct solution was {solution}",
        'msg_restart_quit': "Choose 'r' to restart the game or 'q' to quit the game    ",
        'header_win': "YOU WIN",
        'header_lost': "GAME OVER",
        'fig_texts': {
            'hangman': "Classical hangman figure",
            'flowers': "Field of flowers",
            'chase': "  Police chase"
        }
    },

    'de': {
        'lang_full': "deutsch",
        'msg_avail_diff': "Verfügbare Schwierigkeitsgrade:",
        'difficulties': {
            '1': {
                'function': (random.choice, [["Montag","Dienstag","Mittwoch","Donnerstag","Freitag","Samstag","Sonntag"]],{}),
                'label': "   Wörter - einfach (handerlesen)"
            },
            '2': {
                'function': (random_from_web, ["https://raw.githubusercontent.com/JackShannon/1000-most-common-words/master/1000-most-common-german-words.txt"], {'minlen': 7}),
                'label': "   Wörter - mittel  (1000 häufigste Wörter)"
            },
            '3': {
                'function': (random_from_web, ["https://gist.githubusercontent.com/MarvinJWendt/2f4f4154b8ae218600eb091a5706b5f4/raw/36b70dd6be330aa61cd4d4cdfda6234dcb0b8784/wordlist-german.txt"], {'minlen': 7}),
                'label': "   Wörter - schwer  (Alle möglichen Wörter)"
            },
            '4': {
                'function': (getpass, ["Wähle Lösungswort oder -Satz:  "], {}),
                'label': "   Lösung aus Benutzereingabe"
            },
            '1984': {
                'function': (random_from_web, ["https://raw.githubusercontent.com/MHoffmannAC/Data-Science-Primer/main/doppeldenk"], {}),
                'label': "Doppeldenk-Sätze"
            }
        },
        'msg_inp_diff': "Wähle Schwierigkeitsgrad um ein neues Spiel zu beginnen,\noder ändere die Sprache oder Illustration\n",
        'msg_avail_lett': "Mögliche Buchstaben:",
        'msg_avail_lang': "Mögliche Sprachen:",
        'msg_avail_figs': "Mögliche Illustrationen:",
        'msg_guesses_left': ["Du hast noch einen Fehlversuch",
                             "Du hast noch {num_guesses} Fehlversuche über"],
        'msg_choose_lett': "Bitte wähle einen Buchstaben oder rate die Lösung  ",
        'msg_allowed_input': "Bitte gib einen einzelnen Buchstaben oder die komplette Lösung ein",
        'msg_repeat': "Dieser Buchstabe wurde bereits zuvor geraten",
        'msg_not_solution': "{user_guess} ist leider nicht die Lösung",
        'msg_correct_lett': "Glückwunsch, {user_guess} kommt in der Lösung vor",
        'msg_wrong_lett': "{user_guess} kommt leider nicht in der Lösung vor",
        'msg_success': "Du hast erfolgreich die Lösung {solution} erraten",
        'msg_reveal': "Die korrekte Lösung wäre {solution} gewesen",
        'msg_restart_quit': "Wähle 'r' zum Neustarten oder 'q' zum Beenden des Spiels    ",
        'header_win': "GEWONNEN",
        'header_lost': "VERLOREN",
        'fig_texts': {
            'hangman': "Klassisches Galgenmännchen",
            'flowers': "Blumenfeld",
            'chase': "  Verfolgungsjagd mit Polizei"
        }
    }

# here, new languages can be added. make sure to define ALL dict entries (i.e., copy from an existing one and replace accordingly)

}

In [6]:
# @title define main code run

# main program call:
# 1) show welcome
# 2) loop over game runs until user quits game
#     i)   show screen to select difficulty or language
#     ii)  run single hangman game
#     iii) show result
# 3) show end screen
def run_hangman():
  # some settings. currently not that usefull, but maybe handy in case one adds more features
  settings = {'language': "en",
              'difficulty': 0,
              'figure': 'hangman',
              'figlet': "ansi_regular"  # currently not changable by user
              }
  print(Figlet(font=settings['figlet']).renderText("WELCOME"))
  sleep(2)
  keep_running = True
  # main loop
  while(keep_running):
    # screen to choose language and difficulty
    keep_running = show_welcome_screen(settings)
    # end if user wants to quit (currently a hidden feature, try "quit!" in inputs)
    if(not keep_running):
      break
    # main game call. returns outcome and correct solution
    successful, solution = run_game(settings)
    # show result and ask user whether to quit or to restart
    keep_running = show_result_screen(successful, solution, settings)
  #
  clear_output(wait=True)
  print("")
  print(Figlet(font=settings['figlet']).renderText("GOODBYE"))

## Place to run the game

### note: for a smoother user-experience, make sure the code part above is hidden or switch to fullscreen output during the game

In [ ]:
run_hangman()